# Libraries Import

## Regular Libraries

In [1]:
import numpy as np
import pandas as pd
import os

## Auxiliary Library

In [2]:
from glob import glob

from tqdm import tqdm

import random

## Data Visualization Library

In [3]:
import matplotlib.pyplot as plt

In [4]:
import tensorflow as tf
from transformers import *

In [5]:
import torch
torch.cuda.empty_cache()
from torch.utils.data import Dataset,DataLoader
from transformers import RobertaTokenizerFast, RobertaForTokenClassification

import pytorch_lightning as pl

import pdb
from torch import cuda
from sklearn.metrics import accuracy_score

MODEL_NAME = "../input/roberta-base"

---

# Data Extraction

## Main Data

In [6]:
train = pd.read_csv('../input/feedback-prize-2021/train.csv')
sample_submission = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')

In [7]:
print(train.shape)
train.head()

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [8]:
print(f'Training data consists of {train.shape[0]} annotaions')

Training data consists of 144293 annotaions


In [9]:
print(train["predictionstring"].nunique())
print("------------------------------------")
print(train["discourse_type_num"].nunique())
print(train["discourse_type_num"].unique())
print("------------------------------------")
print(train["discourse_type"].unique())

50355
------------------------------------
44
['Lead 1' 'Position 1' 'Evidence 1' 'Evidence 2' 'Claim 1' 'Evidence 3'
 'Evidence 4' 'Claim 2' 'Evidence 5' 'Concluding Statement 1'
 'Counterclaim 1' 'Rebuttal 1' 'Claim 3' 'Claim 4' 'Claim 5' 'Claim 6'
 'Claim 7' 'Counterclaim 2' 'Rebuttal 2' 'Counterclaim 3' 'Rebuttal 3'
 'Evidence 6' 'Lead 2' 'Counterclaim 4' 'Counterclaim 5' 'Counterclaim 6'
 'Evidence 7' 'Claim 8' 'Evidence 8' 'Concluding Statement 2' 'Rebuttal 4'
 'Rebuttal 5' 'Claim 9' 'Position 2' 'Claim 10' 'Claim 11' 'Claim 12'
 'Evidence 9' 'Concluding Statement 3' 'Concluding Statement 4'
 'Evidence 10' 'Evidence 11' 'Rebuttal 6' 'Evidence 12']
------------------------------------
['Lead' 'Position' 'Evidence' 'Claim' 'Concluding Statement'
 'Counterclaim' 'Rebuttal']


In [10]:
train["discourse_len"] = train["discourse_text"].apply(lambda x:len(x.split()))

In [11]:
train["predictionstring_len"] = train["predictionstring"].apply(lambda x:len(x.split()))

In [12]:
train["len_predict"] = 0

In [13]:
for i in range(0,len(train["discourse_len"])):
    if train["discourse_len"][i] == train["predictionstring_len"][i]:
        train["len_predict"][i] = 0
    else:
        train["len_predict"][i] = 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
train["len_predict"].sum()

468

In [15]:
train.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,discourse_len,predictionstring_len,len_predict
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,44,44,0
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,15,15,0
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,16,16,0
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,63,63,0
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,24,24,0


In [16]:
#test_names, test_texts = [], []
#for f in tqdm(list(os.listdir('../input/feedback-prize-2021/test'))):
#    test_names.append(f.replace('.txt', ''))
#    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r').read())
#test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
#
#test_texts.head()

In [17]:
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})

train_text_df.head()

100%|██████████| 15594/15594 [00:49<00:00, 313.81it/s]


,id,text
0,62C57C524CD2,I think we should be able to play in a sport i...
1,80667AD3FFD8,Some schools require summer projects for stude...
2,21868C40B94F,Driverless cars have been argued and talked ab...
3,87A6EF3113C6,"The author of ""The Challenge of Exploring Venu..."
4,24687D08CFDA,"Wow, from the mar really look like humans face..."


In [18]:
train_df = train.copy()
all_entities = []
for i in tqdm(train_text_df.iterrows()):
    total = i[1]['text'].split(' ').__len__()
    start = -1
    entities = []
    for j in train_df[train_df['id'] == i[1]['id']].iterrows():
        discourse = j[1]['discourse_type']
        list_ix = j[1]['predictionstring'].split(' ')
        ent = [f"I-{discourse}" for ix in list_ix]
        ent[0] = f"B-{discourse}"
        ds = int(list_ix[0])
        de = int(list_ix[-1])
        if start < ds-1:
            ent_add = ['O' for ix in range(int(ds-1-start))]
            ent = ent_add + ent
        entities.extend(ent)
        start = de
    if len(entities) < total:
        ent_add = ["O" for ix in range(total-len(entities))]
        entities += ent_add
    else:
        entities = entities[:total]
    all_entities.append(entities)

15594it [05:40, 45.84it/s]


In [19]:
train_text_df['entities'] = all_entities
train_text_df.head()

,id,text,entities
0,62C57C524CD2,I think we should be able to play in a sport i...,"[B-Position, I-Position, I-Position, I-Positio..."
1,80667AD3FFD8,Some schools require summer projects for stude...,"[B-Position, I-Position, I-Position, I-Positio..."
2,21868C40B94F,Driverless cars have been argued and talked ab...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
3,87A6EF3113C6,"The author of ""The Challenge of Exploring Venu...","[B-Position, I-Position, I-Position, I-Positio..."
4,24687D08CFDA,"Wow, from the mar really look like humans face...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."


In [20]:
config = {'model_name': '/kaggle/input/roberta-base/',
         'max_length': 512,
         'train_batch_size':8,
         'valid_batch_size':8,
         'epochs':3,
         'learning_rate':1e-05,
         'max_grad_norm':10,
         'device': 'cuda' if cuda.is_available() else 'cpu'}

In [21]:
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [22]:
print(labels_to_ids)
print(ids_to_labels)

{'O': 0, 'B-Lead': 1, 'I-Lead': 2, 'B-Position': 3, 'I-Position': 4, 'B-Claim': 5, 'I-Claim': 6, 'B-Counterclaim': 7, 'I-Counterclaim': 8, 'B-Rebuttal': 9, 'I-Rebuttal': 10, 'B-Evidence': 11, 'I-Evidence': 12, 'B-Concluding Statement': 13, 'I-Concluding Statement': 14}
{0: 'O', 1: 'B-Lead', 2: 'I-Lead', 3: 'B-Position', 4: 'I-Position', 5: 'B-Claim', 6: 'I-Claim', 7: 'B-Counterclaim', 8: 'I-Counterclaim', 9: 'B-Rebuttal', 10: 'I-Rebuttal', 11: 'B-Evidence', 12: 'I-Evidence', 13: 'B-Concluding Statement', 14: 'I-Concluding Statement'}


In [23]:
tokenizer = RobertaTokenizerFast.from_pretrained(MODEL_NAME)

model = RobertaForTokenClassification.from_pretrained(config['model_name'], num_labels=len(output_labels))

Some weights of the model checkpoint at /kaggle/input/roberta-base/ were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at /kaggle/input/roberta-base/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on

In [24]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index]
        word_labels = self.data.entities[index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
#                              is_pretokenized=True, 
#                                   is_split_into_words=True,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True,
                            max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
#         pdb.set_trace()
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
#         print(len(sentence), len(labels))
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
#             print(idx)
            if mapping[0] != 0 and mapping[0] != encoding['offset_mapping'][idx-1][1]:
            # overwrite label
#             pdb.set_trace()
#             print(mapping)
#             print(encoded_labels.shape, len(labels), idx, i)
                try:
                    encoded_labels[idx] = labels[i]
                except:
                    pass
                i += 1
            else:
                if idx==1:
    #                 print(idx)
                    encoded_labels[idx] = labels[i]
                    i += 1
        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

    def __len__(self):
        return self.len
                                  

In [25]:
data = train_text_df[['text', 'entities']]
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, config['max_length'])
testing_set = dataset(test_dataset, tokenizer, config['max_length'])

FULL Dataset: (15594, 2)
TRAIN Dataset: (12475, 2)
TEST Dataset: (3119, 2)


In [26]:
train_params = {'batch_size': config['train_batch_size'],
                'shuffle': True,
                'num_workers': 1,
                'pin_memory':True
                }

test_params = {'batch_size': config['valid_batch_size'],
                'shuffle': True,
                'num_workers': 1,
                'pin_memory':True
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [27]:
device = config['device']
device

'cuda'

In [28]:
import gc

gc.collect()
torch.cuda.empty_cache()